To train this agent, click **Runtime** > **Run all**. Make sure you've enabled a free Tesla T4 GPU!

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://art.openpipe.ai"><img src="https://github.com/openpipe/art/raw/main/assets/Documentation_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [GitHub](https://github.com/openpipe/art).

</div>

<a href="https://art.openpipe.ai/"><img src="https://github.com/openpipe/art/raw/main/assets/Header_separator.png" height="5"></a>

This notebook shows how to train a Qwen 2.5 3B model to play tic tac toe. It will demonstrate how to set up a multi-turn agent, how to train it, and how to evaluate it.

Completions and metrics will be logged to Weights & Biases.


### Installation


In [ ]:
# Portions adapted from Unsloth Notebooks (https://github.com/unslothai/notebooks)
# Copyright (c) Unsloth contributors.
# License: GNU LGPL v3.0.
# Modifications by OpenPipe:
# - switched to uv
# - changed vllm/triton pinning logic
# - added protobuf pins
# See /licenses/LGPL-3.0.txt and /licenses/GPL-3.0.txt for full text.

%%capture
import os

if "COLAB_" not in "".join(os.environ.keys()):
    !uv pip install openpipe-art[backend]==0.4.11 --prerelease allow --no-cache-dir
else:
    try:
        import numpy

        get_numpy = f"numpy=={numpy.__version__}"
    except:
        get_numpy = "numpy"
    try:
        import subprocess

        is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except:
        is_t4 = False
    get_vllm, get_triton = (
        ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm", "triton")
    )
    !uv pip install --upgrade \
        openpipe-art[backend]==0.4.11 protobuf==5.29.5 {get_vllm} {get_numpy} --prerelease allow --no-cache-dir
    !uv pip install -qqq {get_triton}

In [ ]:
!pip install git+https://github.com/PrimeIntellect-ai/prime-cli.git
import os
os.environ["PRIME_API_KEY"] = "PLACHOLDER_KEY"
!prime env install m8ngotree/sudoku
import verifiers as vf

In [ ]:
!pip install "verifiers[all]"

### Environment Variables

Later on in the notebook, we'll be creating a model that can automatically logs metrics and chat completions to Weights & Biases. In order to do so, you'll need to provide your Weights & Biases API key as an environment variable.


In [ ]:
import os

# Optional
WANDB_API_KEY = ""
if WANDB_API_KEY:
    os.environ["WANDB_API_KEY"] = WANDB_API_KEY

### Agentic Environment

<a name="Environment"></a>

ART allows your agent to learn by interacting with its environment. In this example, we'll create an environment in which the agent can play tic tac toe.

Feel free to read as much or as little of this section's code as you'd like. The important thing to understand is that we're defining the rules of this agent's environment. In many cases, this will already be defined by the task you're trying to solve, but if you need to define a custom environment, this is how you do it.


In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("sapientinc/sudoku-extreme")


In [ ]:
# my_math_env.py
import verifiers as vf

def load_environment(**kwargs):
    """Load and configure the environment."""
    # 1. Load dataset
    dataset = ds

    # 2. Configure parser
    parser = vf.ThinkParser()

    # 3. Define reward functions -- can automatically reference:
    # - parser, prompt, completion, answer, state , task, info
    def correct_answer(parser, completion, answer):
        response = parser.parse_answer(completion) or ''
        return 1.0 if response.strip() == answer.strip() else 0.0

    # 4. Create rubric
    rubric = vf.Rubric(
        funcs=[correct_answer, parser.get_format_reward_func()],
        weights=[1.0, 0.2]
    )

    # 5. Return configured environment
    return vf.SingleTurnEnv(
        dataset=dataset,
        system_prompt="Think step-by-step, then give your answer.",
        parser=parser,
        rubric=rubric,
        **kwargs  # Pass through additional arguments
    )

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play 2048. We'll use a Qwen 2.5 3B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of.


In [ ]:
from dotenv import load_dotenv

import art
from art.local import LocalBackend

load_dotenv()

random.seed(42)

backend = LocalBackend(path="./.art")

### Creating a Model

Now that we've defined the rules of our environment, we can create a model that will learn to play tic tac toe. We'll use a Qwen 2.5 3B model for this example. The `name` parameter will be associated with a wandb run, and the `base_model` parameter is the model that we'll be training a LoRA on top of.


In [ ]:
import os

model = art.TrainableModel(
    name="001-script",
    project="tic-tac-toe",
    base_model="Qwen/Qwen2.5-3B-Instruct",
)
await model.register(backend)

In [ ]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct")

In [ ]:
import verifiers as vf

# 1. Create environment
env = vf.load_environment("sudoku")

# 2. Load model
model, tokenizer = vf.get_model_and_tokenizer("Qwen/Qwen2.5-1.5B-Instruct")

# 3. Configure training
args = vf.grpo_defaults(run_name="my-experiment")

# 4. Train
trainer = vf.GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    env=env,
    args=args,
)
trainer.train()

### Defining a Rollout

<a name="Rollout"></a>

A rollout is a single episode of an agent performing its task. It generates one or more trajectories, which are lists of messages and choices.

In this example, the rollout function generates a game of tic tac toe, and the agent plays it until the game is finished. It then returns a trajectory which contains all the `system` and `user` messages presented to the agent, as well as all the `choices` that the agent made.

When the game is finished the `reward` for the agent's performance is calculated based on whether the agent won, lost, drew, or errored, which is then assigned to the trajectory.

This rollout function will be called many times in parallel during each step of the training loop.


In [ ]:
from verifiers.types import Messages, State
from typing import Tuple

class MyGameEnv(vf.MultiTurnEnv):

    async def env_response(self, messages: Messages, state: State) -> Tuple[Messages, State]:
        """Define how the environment responds."""
        # Get the last message from the assistant
        last_msg = messages[-1]
        if last_msg["role"] == "assistant":
            player_action = last_msg["content"]
        else:
            return [], state  # No response if not assistant message

        # Check game state
        if self.is_game_over(state):
            response = [{"role": "user", "content": "Game over!"}]
            state["done"] = True
            return response, state

        # Update game state
        state = self.update_state(state, player_action)
        feedback = self.get_game_feedback(state)

        # Return list of ChatMessage dicts
        response = [{"role": "user", "content": feedback}]
        return response, state

def load_environment(**kwargs):
    return MyGameEnv(dataset=ds, **kwargs)

<a name="Loop"></a>

### Training Loop

The training loop is where the magic happens. For each of the 100 steps defined below, the rollout function will be called 200 times in parallel. This means that 200 games will be played at once. Each game will produce a trajectory, which will be used to update the model.

The `gather` step will wait for all of the trajectories to be generated, then it will delete all but the most recent checkpoint and train the model on the new trajectories.

Inference will be blocked until the training is complete.


In [ ]:
TRAINING_STEPS = 2
ROLLOUTS_PER_STEP = 48
LEARNING_RATE = 5e-5

for i in range(await model.get_step(), TRAINING_STEPS):
    train_groups = await art.gather_trajectory_groups(
        (
            art.TrajectoryGroup(
                rollout(model, TicTacToeScenario(step=i))
                for _ in range(ROLLOUTS_PER_STEP)
            )
            for _ in range(1)
        ),
        pbar_desc="gather",
    )
    await model.delete_checkpoints()
    await model.train(train_groups, config=art.TrainConfig(learning_rate=LEARNING_RATE))

### Using the Model

Just like that, you've trained an agent to play tic tac toe! Now it's time to use your model outside of ART, in the wild! The easiest way to do that is to load it from disk, where it was saved after each training step, and either run inference on it locally or upload it to a central hub like HuggingFace.

Check out the code below for small demo of the model you just trained playing tic tac toe!


In [ ]:
import os
from pathlib import Path

# example: .art/tic-tac-toe/models/002/checkpoints/0003
lora_model_path = (
    f".art/{model.project}/models/{model.name}/checkpoints/{await model.get_step():04d}"
)

if Path(lora_model_path).exists():
    import torch
    from unsloth import FastLanguageModel

    print(f"loading model from {lora_model_path}\n")

    peft_model, tokenizer = FastLanguageModel.from_pretrained(
        model_name=lora_model_path,
        max_seq_length=16384,
        dtype=torch.bfloat16,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(peft_model)

    game = generate_game()
    move_number = 0

    messages = [
        {
            "role": "system",
            "content": f"You are a tic-tac-toe player. You are playing against an opponent. Always choose the move most likely to lead to an eventual win. Return your move as an XML object with a single property 'move', like so: <move>A1</move>. Optional moves are 'A1', 'B3', 'C2', etc. You are the {game['agent_symbol']} symbol.",
        },
    ]

    if game["agent_symbol"] == "o":
        starting_opponent_move = get_opponent_move(game)
        game["board"][starting_opponent_move[0]][starting_opponent_move[1]] = game[
            "opponent_symbol"
        ]

    while check_winner(game["board"]) is None:
        rendered_board = render_board(game)
        messages.append({"role": "user", "content": rendered_board})

        inputs = tokenizer.apply_chat_template(
            messages, return_tensors="pt", add_generation_prompt=True
        ).to("cuda")

        content = ""

        def get_completion() -> str:
            with torch.no_grad():
                outputs = peft_model.generate(
                    input_ids=inputs,
                    max_new_tokens=100,
                    do_sample=True,
                    temperature=0.7,
                    top_p=0.9,
                )
                return tokenizer.decode(
                    outputs[0][inputs.shape[1] :], skip_special_tokens=True
                )

        try:
            content = get_completion()
        except Exception as e:
            print("caught exception generating chat completion", e)
            raise e

        messages.append({"role": "assistant", "content": content})

        try:
            apply_agent_move(game, content)
            move_number += 1
        except ValueError as e:
            print(f"Invalid move on move {move_number}: {content}")
            print(f"Reason: {e}")
            continue

        # print the board every move
        print(f"\nmove {move_number}")
        print(f"board:\n{rendered_board}")
        print(f"agent move: {content}")
        print(f"updated board:\n{render_board(game)}")

        if check_winner(game["board"]) is not None:
            break
        move_number += 1

        opponent_move = get_opponent_move(game)
        game["board"][opponent_move[0]][opponent_move[1]] = game["opponent_symbol"]

    winner = check_winner(game["board"])

    print(f"game finished in {move_number} moves")

    if winner == game["agent_symbol"]:
        print("game won! 💪")
    elif winner == game["opponent_symbol"]:
        print("game lost! 😢")
    elif winner == "draw":
        print("draw! 🤷‍♂️")

    print(f"final board:\n\n{render_board(game)}")

NameError: name 'model' is not defined

<div class="align-center">
<a href="https://github.com/openpipe/art"><img src="https://github.com/openpipe/art/raw/main/assets/ART_pill.png" height="50"></a>
<a href="https://discord.gg/zbBHRUpwf4"><img src="https://github.com/openpipe/art/raw/main/assets/Discord.png" height="50"></a>
<a href="https://openpipe.ai/blog/art-e-mail-agent"><img src="https://github.com/openpipe/art/raw/main/assets/ART_E_pill.png" height="50"></a>

Questions? Join the Discord and ask away! For feature requests or to leave a star, visit our [Github](https://github.com/openpipe/art).

</div>
